## Feature Selection using Decision Tree
###### Step 1: Data Normalization
###### Step 2: Shuffled model (permutation tests) for 100 times, with random labels
###### Step 3: 10-fold validation for 100 DT models (generating 100*10 shuffled variable importances)
###### Step 4: calculate mean values of variable importances (generating 10 mean shuffled variable importances, each feature 10 shuffled importances)
###### Step 5: 10-fold validation for the real model (generating 10 real variable importances, each feature 10 real importances)
###### Step 6: Significance testing for shuffled and real importances (Wilcoxon signed-rank test)

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline

# PRS dataset
dataset = pd.read_pickle('datafile/PRS_dataset.pkl')
dataset.head()

,Is_anxiety,446,447,448,449,450,451,452,453,454,...,848,849,850,851,852,853,854,855,856,857
1000023,0,0.000507,0.00242,-0.000185,-0.000145,-0.000371,-0.000873,0.012805,0.000115,0.000151,...,0.025738,-0.001572,0.014031,-0.000354,-0.000001,0.000654,-0.002815,-0.000183,0.01424,-0.00352
1000030,1,0.000276,0.001576,0.000038,-0.000381,-0.000399,-0.004763,0.014032,0.000109,0.000093,...,0.014733,-0.000623,0.005552,-0.000244,0.00031,-0.00019,0.000197,-0.00005,0.01234,-0.00579
1000041,0,0.000649,-0.000098,-0.000086,0.00009,-0.000678,-0.004804,0.00727,-0.000024,0.000109,...,0.011649,-0.000921,0.010997,-0.000557,0.000115,-0.000354,-0.01137,-0.000419,0.006975,-0.004178
1000059,0,0.000585,0.000814,-0.000066,-0.000621,0.000105,0.00187,0.002563,0.000106,0.000097,...,0.022186,-0.000923,0.01562,-0.000159,0.000229,-0.000769,0.006247,-0.000164,0.001851,-0.001964
1000062,0,0.000214,0.00149,0.000117,-0.000153,-0.000603,-0.005913,0.012513,0.000089,0.000109,...,0.022118,-0.000425,0.007184,0.000349,0.000295,0.001721,0.011161,-0.0002,0.010253,-0.006825


In [4]:
# step 1
def normalization(data):
    _range = np.max(abs(data))
    return data / _range

nor_feature = dataset.drop(['Is_anxiety'], axis=1)
nor_feature = normalization(nor_feature)
nor_feature.head()

c:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


,446,447,448,449,450,451,452,453,454,455,...,848,849,850,851,852,853,854,855,856,857
1000023,0.50716,0.489223,-0.268062,-0.115851,-0.119164,-0.064759,0.30606,0.272503,0.47023,0.037408,...,0.402434,-0.426125,0.464718,-0.22394,-0.00059,0.158681,-0.098135,-0.237317,0.342466,-0.261778
1000030,0.27551,0.318646,0.054593,-0.303897,-0.128181,-0.353385,0.335409,0.25835,0.289395,0.029372,...,0.23036,-0.168929,0.183895,-0.154455,0.340093,-0.046103,0.006876,-0.064666,0.296776,-0.430573
1000041,0.649187,-0.019783,-0.124265,0.071839,-0.217783,-0.356422,0.173778,-0.057635,0.338935,-0.113878,...,0.182137,-0.249577,0.364249,-0.352415,0.126125,-0.085873,-0.396317,-0.543573,0.167738,-0.310705
1000059,0.585169,0.164566,-0.095402,-0.495318,0.033831,0.138735,0.061268,0.250783,0.302471,0.299225,...,0.3469,-0.250089,0.517354,-0.100632,0.251434,-0.186582,0.217754,-0.213378,0.04452,-0.146034
1000062,0.214057,0.301174,0.169086,-0.122194,-0.193515,-0.43875,0.299083,0.209926,0.33983,-0.220002,...,0.345838,-0.115133,0.237952,0.22093,0.323542,0.417538,0.389024,-0.258967,0.246564,-0.507541


In [ ]:
# step 2-3